In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time




In [16]:
# القوائم لتخزين البيانات
titles = []
authors = []
years = []



In [17]:

# التكرار على الصفحات
for page in range(1, 50):
    url = f"https://openlibrary.org/search?q=trending_score_hourly_sum%3A%5B1+TO+%2A%5D+-subject%3A%22content_warning%3Acover%22+language%3Aeng&sort=trending&page={page}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"⚠️ تخطي الصفحة {page} (لم يتم الوصول إليها)")
        continue

    soup = BeautifulSoup(response.content, "html.parser")
    books = soup.find_all("li", class_="searchResultItem")

    for book in books:
        # العنوان
        title_tag = book.find("a", class_="results", itemprop="url", href=True)
        title = title_tag.text.strip() if title_tag else "No title"

        # المؤلف
        author_tag = book.find("span", class_="bookauthor")
        author = author_tag.text.strip().replace("by ", "") if author_tag else "Unknown"

        # السنة
        year_tag = book.find("span", class_="resultDetails")
        if year_tag:
            first_span = year_tag.find("span")
            if first_span and "First published in" in first_span.text:
                year = first_span.text.strip().replace("First published in", "").strip()
            else:
                year = "N/A"
        else:
            year = "N/A"
        titles.append(title)
        authors.append(author)
        years.append(year)


    time.sleep(1)  # احترام تأخير robots.txt

In [18]:

# إنشاء DataFrame
df = pd.DataFrame({
    "Title": titles,
    "Author": authors,
    "Year": years,

})


In [19]:

# حفظ البيانات كملف CSV
df.to_csv("trending_books.csv", index=False, encoding="utf-8")
